<br><br><br><br><br><h1 style="font-size:4em;color:#2467C0">Welcome to Week 3</h1><br><br><br>


<p style="line-height:31px;">This document provides a running example of completing the Week 3 assignment : </p>
<ul class="simple">
<li style="line-height:31px;">You can then copy paste the commands below in pySpark. </li>
<li style="line-height:31px;">To open pySpark, type pyspark and hit enter</li>
<br>
Finally, make sure that your working directory contains the data file(s) (.csv) for the fastest support. <br><br> We recommend workng in your home directory (type cd ~ in your terminal). Please run any scripts using your terminal for proper settings
<br>

<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">
<br><br><br>
To run these commands in Cloudera VM: first run the setup script: setupWeek3.sh</p>

In [5]:
from pyspark.sql import SQLContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

In [ ]:
pip install pyspark

<br><br>

<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Import Data
</p>

 

<br><br>
<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">First let us read the contents of the file w3_clustering.csv. The following commands read in the CSV file <br> in a table format. <br><br>


If you want to run this command on your machine: Note that you must change the path in the following <br>
command to 'w3_clustering.csv' to point to the location on your local machine.
</p>

</div>

<br><br><br><br>

In [6]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
data = sqlContext.read.load('w3_clustering.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Let us display the number of lines in the data:

</p>

<br><br>

In [7]:
data.count()

709

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

What does the data contain ?
</p>

 
<br><br>

In [8]:
data

DataFrame[totalAdClicks: int, revenue: double]

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Statistics about the data:
</p>

 
<br><br>

In [9]:
data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
totalAdClicks,709,32.208744710860366,16.384120817042554,1,73
revenue,709,44.64880112834979,44.944528765285284,1.0,278.0


<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Drop all rows with NULL or NaN values

</p>

 
<br><br>

In [23]:
data = data.na.drop()

AttributeError: 'DataFrame' object has no attribute 'na'

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Let us look at the column names:

</p>

 
 
<br><br>

In [11]:
data.columns

['totalAdClicks', 'revenue']

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Use VectorAssembler to gather all the features:

</p>

 
<br><br>

In [28]:
featuresUsed = ['isHit', 'price']
assembler = VectorAssembler(inputCols=featuresUsed, outputCol="features_unscaled")
assembled = assembler.transform(data)

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Scale the features using StandardScaler:

</p>

 
<br><br>

In [29]:
scaler = StandardScaler(inputCol="features_unscaled", outputCol="features", withStd=True, withMean=True)
scalerModel = scaler.fit(assembled)
scaledData = scalerModel.transform(assembled)

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Select the features column make the data persist:

</p>

 
<br><br>

In [30]:
scaledData = scaledData.select("features")
scaledData.persist()

DataFrame[features: vector]

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

We can now perform KMeans clustering to generate 2 clusters:
</p>

 
<br><br>

In [37]:
kmeans = KMeans(k=3, seed=1)
model = kmeans.fit(scaledData)
transformed = model.transform(scaledData)

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Print the center of these two clusters:

</p>

 
<br><br>

In [38]:
centers = model.clusterCenters()
centers

[array([-0.32965623, -0.36089697]),
 array([0.10815052, 1.97281387]),
 array([ 2.27468978, -0.0926773 ])]

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Analyze center of these two clusters:

</p>

 
<br><br>


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Each array denotes the center for a cluster:<br><br>
One Cluster is centered at   ... array([ 0.84174521,  0.51884657])<br>
Other Cluster is centered at   ... array([-0.79780796, -0.49176392])</p>

<br><br>

<p style="line-height:31px;"> First number (field1) in each array refers to scaled verson of the number of ad-clicks and the second number (field2) is the scaled version of the revenue per user.

Compare the 1st number of each cluster to see how differently users in each cluster behave when it comes to clicking ads.

Compare the 2nd number of each cluster to see how differently users in each cluster behave when it comes to buying stuff. 

</p><br><br>

<p style="line-height:31px;">In one cluster, in general, players click on ads much more often and spend more money on in-app purchases. Assuming that Eglence Inc. gets paid for showing ads and for hosting in-app purchase items, we can use this information to increase game's revenue by increasing the prices for ads we show to the frequent-clickers, and charge higher fees for hosting the in-app purchase items shown to the higher revenue generating buyers.</p>

<br><br>
<p style="line-height:31px;"> <b> Note: </b>  This analysis requires you to compare the cluster centers and find any ‘significant’ differences in the corresponding feature values of the  centers. The answer to this question will depend on the features you have chosen. <br><br> Some features help distinguish the clusters remarkably while others may not tell you much. At this point, if you don’t find clear distinguishing patterns, perhaps re-running the clustering model with different numbers of clusters and revising the features you picked would be a good idea. </p>

</div>


# Training Data Set Creation

The training data set used for this analysis is shown below (first 5 lines):
Read of the following files game-clicks.csv and buy-clicks.csv
Feature Selection of the selected columns

In [19]:
data_game_clicks = sqlContext.read.load('game-clicks.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')
gameclicks_df = data_game_clicks.toPandas()

data_buy_clicks = sqlContext.read.load('buy-clicks.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')
buyclicks_df = data_buy_clicks.toPandas()

In [21]:
user_purchases = buyclicks_df[['userId','price']]
user_hits = gameclicks_df[['userId','isHit']]
# Sum the target columns per User Id
hits_per_user = user_hits.groupby('userId').sum()
revenue_per_user = user_purchases.groupby('userId').sum()
# Merge the datasets into one for the analysis
combined_df = hits_per_user.merge(revenue_per_user, on='userId')

In [24]:
data = combined_df.dropna()

In [26]:
data = sqlContext.createDataFrame(data)


In [41]:
transformed.toPandas().shape

(546, 2)

In [42]:
gameclicks_df.head()

,timestamp,clickId,userId,userSessionId,isHit,teamId,teamLevel
0,2016-05-26 15:06:55,105,1038,5916,0,25,1
1,2016-05-26 15:07:09,154,1099,5898,0,44,1
2,2016-05-26 15:07:14,229,899,5757,0,71,1
3,2016-05-26 15:07:14,322,2197,5854,0,99,1
4,2016-05-26 15:07:20,22,1362,5739,0,13,1


In [44]:
buyclicks_df.head()

,timestamp,txId,userSessionId,team,userId,buyId,price
0,2016-05-26 15:36:54,6004,5820,9,1300,2,3.0
1,2016-05-26 15:36:54,6005,5775,35,868,4,10.0
2,2016-05-26 15:36:54,6006,5679,97,819,5,20.0
3,2016-05-26 16:36:54,6067,5665,18,121,2,3.0
4,2016-05-26 17:06:54,6093,5709,11,2222,5,20.0
